In [2]:
#set up your tavily key in plave of your-tavily-key
!python3 -m pip install --upgrade pip
!pip3 install -qU langgraph
!pip3 install langgraph-checkpoint-sqlite
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
import os
import requests
import json
from openai import OpenAI
import httpx
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from langchain.tools import tool
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langgraph.checkpoint.sqlite import SqliteSaver
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults
from cfenv import AppEnv

# connect to tavily search tool - use your tavily api key
os.environ['TAVILY_API_KEY']= "your-tavily-key"
tool = TavilySearchResults(max_results=2)

#define agent state
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
#memory = SqliteSaver.from_conn_string(":memory:")

print(type(tool))
print(tool.name)

class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
# Load CF environment
env = AppEnv()

# configure model
httpx_client = httpx.Client(http2=True, verify=False, timeout=10.0)
# Get bound service "gen-ai-qwen3-ultra"
chat_service = env.get_service(name="gen-ai-qwen3-ultra")
chat_credentials = chat_service.credentials

# Initialize LLM with credentials from cfenv
model = ChatOpenAI(
    temperature=0.9,
    model=chat_credentials["model_name"],
    base_url=chat_credentials["api_base"],
    api_key=chat_credentials["api_key"],
    http_client=httpx_client
)
messages = [HumanMessage(content="What is the weather in Austin?")]
thread = {"configurable": {"thread_id": "1"}}
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], system=prompt, checkpointer=checkpointer)
    for event in abot.graph.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])

/home/vcap/app/.venv/bin/python3: No module named pip
/usr/bin/sh: 1: pip3: not found
/usr/bin/sh: 1: pip3: not found
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json
[AIMessage(content='<think>\nOkay, the user is asking for the weather in Austin. I need to figure out how to get that information. Since I don\'t have a built-in weather function, I should use the available tools. The user provided a search engine function called tavily_search_results_json. I can use that to look up the current weather in Austin. Let me make sure the query is clear. I\'ll search for "current weather in Austin" to get the most accurate and up-to-date information. Alright, let\'s call the function with that query.\n</think>\n\n', additional_kwargs={'tool_calls': [{'id': '', 'function': {'arguments': '{"query":"current weather in Austin"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'co